# cube.link - Data Integration

This notebook demonstrates the **data integration** process starting from an open government data CSV to a RDF cube according to https://cube.link schema.

## Dataset Identifier

- 2023: https://opendata.swiss/de/perma/24885584@bundesamt-fur-statistik-bfs
- 2024: https://opendata.swiss/de/perma/31646008@bundesamt-fur-statistik-bfs

## Read Raw Data

In [2]:
import pandas as pd

df = pd.read_csv('https://dam-api.bfs.admin.ch/hub/api/dam/assets/31646012/master')

display(df.head(10))
display(df.tail(10))

,GEO_NR,GEO_NAME,CLASS_HAB,GEOM_PERIOD,VARIABLE,SOURCE,VALUE_PERIOD,UNIT_VALUE,VALUE,OBS_CONFIDENCE,STATUS
0,CH,Schweiz / Suisse,Total,2023-01-01,tran_t,MFZ,2022,Num,6368579.0,NaN,A
1,CH,Schweiz / Suisse,Total,2023-01-01,voit_tour,MFZ,2022,Num,4721280.0,NaN,A
2,CH,Schweiz / Suisse,Total,2023-01-01,voit_tour_e,MFZ,2022,Num,110751.0,NaN,A
3,CH,Schweiz / Suisse,Total,2023-01-01,voit_livr,MFZ,2022,Num,421014.0,NaN,A
4,CH,Schweiz / Suisse,Total,2023-01-01,tran_lou,MFZ,2022,Num,54700.0,NaN,A
5,CH,Schweiz / Suisse,Total,2023-01-01,moto,MFZ,2022,Num,789794.0,NaN,A
6,CH,Schweiz / Suisse,Total,2023-01-01,tran_aut,MFZ,2022,Num,381791.0,NaN,A
7,CH,Schweiz / Suisse,Total,2023-01-01,txmot,"MFZ, STATPOP",2022,Per,540.0,NaN,A
8,CH,Schweiz / Suisse,Total,2023-01-01,voit_tour_new,IVS,2022,Num,229403.0,NaN,A
9,CH,Schweiz / Suisse,Total,2023-01-01,voit_tour_new_e,IVS,2022,Num,40507.0,NaN,A


,GEO_NR,GEO_NAME,CLASS_HAB,GEOM_PERIOD,VARIABLE,SOURCE,VALUE_PERIOD,UNIT_VALUE,VALUE,OBS_CONFIDENCE,STATUS
2213,6711,Delémont,2,2023-01-01,voit_livr,MFZ,2022,Num,580.00,NaN,A
2214,6711,Delémont,2,2023-01-01,tran_lou,MFZ,2022,Num,83.00,NaN,A
2215,6711,Delémont,2,2023-01-01,moto,MFZ,2022,Num,740.00,NaN,A
2216,6711,Delémont,2,2023-01-01,tran_aut,MFZ,2022,Num,300.00,NaN,A
2217,6711,Delémont,2,2023-01-01,txmot,"MFZ, STATPOP",2022,Per,524.00,NaN,A
2218,6711,Delémont,2,2023-01-01,voit_tour_new,IVS,2022,Num,394.00,NaN,A
2219,6711,Delémont,2,2023-01-01,voit_tour_new_e,IVS,2022,Num,62.00,NaN,A
2220,6711,Delémont,2,2021-07-01,pen_t,"PEND, SE",2017/2021,Num,4545.27,7.3,A
2221,6711,Delémont,2,2021-07-01,pen_tp,"PEND, SE",2017/2021,Num,850.28,17.3,A
2222,6711,Delémont,2,2021-07-01,pen_tim,"PEND, SE",2017/2021,Num,2738.07,9.5,A


## Get Overview of the Data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2223 entries, 0 to 2222
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   GEO_NR          2223 non-null   object 
 1   GEO_NAME        2223 non-null   object 
 2   CLASS_HAB       2223 non-null   object 
 3   GEOM_PERIOD     2223 non-null   object 
 4   VARIABLE        2223 non-null   object 
 5   SOURCE          2223 non-null   object 
 6   VALUE_PERIOD    2223 non-null   object 
 7   UNIT_VALUE      2223 non-null   object 
 8   VALUE           2223 non-null   float64
 9   OBS_CONFIDENCE  513 non-null    float64
 10  STATUS          2223 non-null   object 
dtypes: float64(2), object(9)
memory usage: 191.2+ KB


What are the numbers for the unique values for each column? 

In [4]:
{col: df[col].nunique() for col in df.columns}

{'GEO_NR': 171,
 'GEO_NAME': 171,
 'CLASS_HAB': 7,
 'GEOM_PERIOD': 2,
 'VARIABLE': 13,
 'SOURCE': 4,
 'VALUE_PERIOD': 2,
 'UNIT_VALUE': 2,
 'VALUE': 1659,
 'OBS_CONFIDENCE': 158,
 'STATUS': 1}

## Wide and Long Format

The data is in long format. Meaning basically that the actual measurement data is in one single column (`VALUE`). The other columns give some information about the **dimension** of the data. The most important one is `VARIABLE` which states, what is actually measured.

This is well suited for the cube.link model. However, there is a problem with the measurements units. The cube.link schema allows for only one unit per column but there are two (`Num` and `Per`). We will have to deal with this later.

In [5]:
{col: df[col].unique().tolist() for col in ["VARIABLE", "UNIT_VALUE"]}

{'VARIABLE': ['tran_t',
  'voit_tour',
  'voit_tour_e',
  'voit_livr',
  'tran_lou',
  'moto',
  'tran_aut',
  'txmot',
  'voit_tour_new',
  'voit_tour_new_e',
  'pen_t',
  'pen_tp',
  'pen_tim'],
 'UNIT_VALUE': ['Num', 'Per']}

## Column by Column Analysis

### `GEO_NR` and `GEO_NAME`

Based on the exact same number of unique values for both these columns, we can assume that the `GEO_NR` is a code for the `GEO_NAME`. A little bit of Swiss geography knowledge tells us, that there are different classes of geographic units of observation. There are values for whole of Switzerland, and for single (larger) municipalities. This is great because LINDAS and visualize at the moment can not aggregate data.

### `VARIABLE`

This is a very typic column in CSV. Basically, it is a **dimension** of the data. It tells us what is actually measured. The problem is, if you are not an expert, you would never know what e.g. `voit_tour` or `pen_tim` actually means. This is one of the main reasons why we are doing the whole data integration stuff and why this is so important.

Normally, there is some accompanying documentation to a open government data set. In this case, this data is even machine readable structured data:

In [6]:
# Read the ODS file into a dictionary of DataFrames
dfs = pd.read_excel("https://dam-api.bfs.admin.ch/hub/api/dam/assets/24885589/appendix", sheet_name=None, engine="odf")

# Now `dfs` is a dictionary where the keys are sheet names and the values are DataFrames
for sheet_name, df in dfs.items():
    print(f"Sheet: {sheet_name}")
    display(df.head())  # Display the first few rows of each sheet

Sheet: Information


,Unnamed: 0,Statistik der Schweizer Städte 2023,Statistique des villes suisses 2023
0,NaN,Allgemeine Anmerkungen,Remarques générales
1,Schweizer StädteVilles suisses,Die in dieser Publikation betrachteten Städte ...,Les villes considérées dans cette publication ...
2,Title,"Mobilität und Verkehr, Statistik der Schweizer...","Mobilité et transports, Statistiques des ville..."
3,Notes,"Datenkombinationen, die nicht vorhanden sind, ...",Les croisements inexistants dans les données n...
4,NaN,Hinweis: Die Zuteilung der Fahrzeuge zu den ei...,Remarque: L'attribution des véhicules aux diff...


Sheet: Legend


,GEO_NR,"Die Länder-, Total der Städte, die Gemeindegrössenklasse und Gemeinde BFS-Nummer.- Schweiz: CH- Alle: _T- Gemeindegrössenklasse mit weinger als 10000 Einwohnern: _ST1- Gemeindegrössenklasse mit 10000-14999 Einwohnern: _ST2- Gemeindegrössenklasse mit 15000-19999 Einwohnern: _ST3- Gemeindegrössenklasse mit 20000-49999 Einwohnern: _ST4- Gemeindegrössenklasse mit 50000-99999 Einwohnern: _ST5- Gemeindegrössenklasse mit 100000 Einwohnern und mehr: _ST6","Le pays, le total des villes, le total pour les classes de taille des villes et le numéro OFS des communes. - Suisse: CH- Total: _T- classe de taille des communes de moins de 10000 habitants: _ST1- classe de taille des communes de 10000-14999 habitants: _ST2- classe de taille des communes de 15000-19999 habitants: _ST3- classe de taille des communes de 20000-49999 habitants: _ST4- classe de taille des communes de 50000-99999 habitants: _ST5- classe de taille des communes de 100000 habitants et plus: _ST6"
0,GEO_NAME,Den Namen der Gemeinde oder der Gemeindegrösse...,Le nom de la commune ou de la classe de taille...
1,CLASS_HAB,Die Codes der Gemeindegrössenklasse der Statis...,Les codes des classes de taille des villes de ...
2,GEOM_PERIOD,Das Referenzdatum der Gemeinde. Der Stand der ...,La date de référence de l'entité communale. L'...
3,VARIABLE,Die Codesliste für den Beobachtungsvariablen.,La liste des codes pour les variables observées.
4,SOURCE,Die Codesliste für die Quellen.,La liste des codes pour les sources.


Sheet: CLASS_HAB


...

,CODE,label_de,label_fr
0,MFZ,"BFS, ASTRA – Strassenfahrzeugbestand (MFZ)","OFS, OFROU – Parc des véhicules routiers (MFZ)"
1,PEND,BFS – Pendlermobilität (PEND),OFS – Pendularité (PEND)
2,SE,BFS – Strukturerhebung (SE),OFS – Relevé structurel (RS)
3,IVS,"BFS, ASTRA – Neue Inverkehrsetzungen von Stras...","OFS, OFROU – Nouvelles mises en circulation de..."
4,STATPOP,BFS – Statistik der Bevölkerung und der Hausha...,OFS – Statistique de la population et des ména...


Sheet: UNIT_VALUE


,CODE,label_de,label_fr
0,Num,Anzahl der (Absolute Zahlen),Nombre de (chiffres absolus)
1,Per,"Quote, in Prozent (Relativenzahlen)","Taux, en pourcent (chiffres relatifs)"


Sheet: STATUS


,CODE,label_de,label_fr
0,A,Normaler Wert,Valeur normale
1,N,"Nicht signifikant, wird verwendet um einen Wer...","Non significatif, utilisé pour indiquer une va..."
2,U,Statistisch nur bedingt zuverlässig,Fiabilité statistique relative
3,L,"Entfällt, weil statistisch nicht sicher genug",Non indiqué par manque de fiabilité statistique
4,O,"Entfällt, weil (noch) nicht erhoben oder (noch...",Non indiqué car pas (encore) relevé ou pas (en...


## Build `VARIABLE` Concept

## Integration with pylindas